# Creación de conversaciones HTTP

### Descripción: 
Con este código se va a intentar crear todos los pasos de una conversación HTTP, mediante IA generativa. Se pretende pasar una descripción de esta conversación al modelo y que este sea capaz de crear todos los pasos/paquetes necesarios para replicar una conversación con esas mismas especificaciones

In [ ]:
import random
import re
import pickle
import os 
import openai
from openai import OpenAI
os.environ['OPENAI_API_KEY'] = 'FILL'
client = OpenAI()

from scapy.all import *
from scapy.utils import RawPcapReader, wrpcap
import scapy.all as scapy

from scapy.layers.inet import IP, TCP
from scapy.layers.http import *

openai.__version__

C:\Users\adayd\anaconda3\envs\PAC-GPT\Lib\site-packages\scapy\layers\ipsec.py:469: CryptographyDeprecationWarning: Blowfish has been deprecated
  cipher=algorithms.Blowfish,
C:\Users\adayd\anaconda3\envs\PAC-GPT\Lib\site-packages\scapy\layers\ipsec.py:483: CryptographyDeprecationWarning: CAST5 has been deprecated
  cipher=algorithms.CAST5,


'1.10.0'

## Step 1: Generación de la descipción de una comunicación HTTP
---

<center><img src="https://notes.shichao.io/tcpv1/figure_13-1.png">
    
<center> Esta imagen es genérica, se intentará crear tanto la parte de hanshake como la terminación de la conversación en una primera aproximación. El intercambio de información siguiendo el protocolo HTTP se hace en la parte intermedia de la conversación, post handshake.


In [2]:
system_message = "This is a IPv4 TCP-HTTP conversation summary:\n"
system_message += 'Source: IP="193.24.227.230", port=40301, Window: 4128 // Destination: IP="9.9.9.9", port=80, Window: 256// Others: Host="http://ip.webernetz.net/", Path="", Code=200,  Content_Type="(text/html)"\n\n'

system_message += "x,y = random.sample(range(1, 100), 2)\n"
system_message += 'pkt_1= IP(src="193.24.227.230", dst="9.9.9.9")/TCP(sport=40301, dport=80, flags="S", seq=y, window=4128)\n'
system_message += 'time.sleep(abs(random.gauss(0, 0.03)))\n'
system_message += 'y += len(pkt_1[TCP].payload)+1\n'
system_message += 'pkt_2= IP(src="9.9.9.9", dst="193.24.227.230")/TCP(sport=80, dport=40301, flags="SA", seq=x, ack=y, window=256)\n'
system_message += 'time.sleep(abs(random.gauss(0, 0.03)))\n'
system_message += 'x += len(pkt_2[TCP].payload)+1\n'
system_message += 'pkt_3= IP(src="193.24.227.230", dst="9.9.9.9")/TCP(sport=40301, dport=80, flags="A", seq=y, ack=x, window=4128)\n\n'
system_message += 'time.sleep(abs(random.gauss(0, 0.03)))\n'
system_message += 'y += len(pkt_3[TCP].payload)\n'

system_message += 'pkt_4= IP(src="193.24.227.230",  dst="9.9.9.9") / TCP(sport=40301, dport=80, flags="PA", seq=y, ack=x, window=8192) / HTTP() / HTTPRequest(Method ="GET", Http_Version="HTTP/1.1", Host="ip.webernetz.net", Path="/")\n'
system_message += 'time.sleep(abs(random.gauss(0, 0.03)))\n'
system_message += 'y += len(pkt_4[TCP].payload)\n'
system_message += 'pkt_5= IP(src="9.9.9.9", dst="193.24.227.230")/TCP(sport=80, dport=40301, flags="A", seq=x, ack=y, window=4128)\n\n'
system_message += 'time.sleep(abs(random.gauss(0, 0.03)))\n'
system_message += 'x += len(pkt_5[TCP].payload)\n'
system_message += 'pkt_6= IP(src="9.9.9.9",  dst="193.24.227.230") / TCP(sport=80, dport=40301, flags="PA", seq=x, ack=y, window=8192) / HTTP() / HTTPResponse(Http_Version= "HTTP/1.1", Status_Code= "200", Reason_Phrase= "OK", Content_Type="text/html") / "<HTML><BODY><H!>Hello World!</H1></BODY></HTML>"\n'
system_message += 'time.sleep(abs(random.gauss(0, 0.03)))\n'
system_message += 'x += len(pkt_6[TCP].payload)\n'
system_message += 'pkt_7= IP(src="193.24.227.230", dst="9.9.9.9")/TCP(sport=40301, dport=80, flags="A", seq=y, ack=x, window=4128)\n\n'
system_message += 'time.sleep(abs(random.gauss(0, 0.03)))\n'
system_message += 'y += len(pkt_7[TCP].payload)\n'

system_message += 'pkt_8= IP(src="193.24.227.230", dst="9.9.9.9")/TCP(sport=40301, dport=80, flags="FA", seq=y, ack=x, window=4128)\n'
system_message += 'time.sleep(abs(random.gauss(0, 0.03)))\n'
system_message += 'y += len(pkt_8[TCP].payload)\n'
system_message += 'pkt_9= IP(src="9.9.9.9", dst="193.24.227.230")/TCP(sport=80, dport=40301, flags="A", seq=x, ack=y, window=256)\n'
system_message += 'time.sleep(abs(random.gauss(0, 0.03)))\n'
system_message += 'x += len(pkt_9[TCP].payload)\n'
system_message += 'pkt_10= IP(src="9.9.9.9", dst="193.24.227.230")/TCP(sport=80, dport=40301, flags="FA", seq=x, ack=y, window=256)\n'
system_message += 'time.sleep(abs(random.gauss(0, 0.03)))\n'
system_message += 'x += len(pkt_10[TCP].payload)\n'
system_message += 'pkt_11= IP(src="193.24.227.230", dst="9.9.9.9")/TCP(sport=40301, dport=80, flags="A", seq=y, ack=x, window=4128)\n\n'
system_message += 'y += len(pkt_11[TCP].payload)\n'

system_message += "Variable names in the HTTP definition are extremely important, use the exact same word. For example, use Http_Version, not http_version, use Method, not method.\n\n"
system_message += "In the HTTP response, if the variable code != 200, don´t use the Content_Type variable\n"
system_message += "Use the same dport for HTTPRequest (dport=80) and sport for HTTPResponse (sport=80), as this means we are creating a HTTP conection and using another port like 22 might make it seem like SSH traffic\n\n"
system_message += 'Pay special attention to the sequence and ack numbers so they make sense for a TCP conversation. The must be continuous throgh the conversation. Here you have a little explanation of how they work:\n'
system_message += 'The Sequence Number (SEQ) marks the starting byte of each segment sent. It increments by the number of bytes in the payload of each outgoing segment, reflecting the continuous flow of data.\n'
system_message += 'The Acknowledgment Number (ACK) acknowledges the receipt of data. It corresponds to the next expected byte the receiver anticipates from the sender. As segments arrive at the receiver, the ACK number adjusts to reflect the highest contiguous sequence number received.\n'
system_message += 'So, SEQ numbers increase with outgoing data, while ACK numbers advance with incoming acknowledgments, ensuring synchronized data exchange in a TCP connection.\n\n'

system_message += "Take your time (few seconds) to generate the commands and compare your proposed solution with real HTTP functioning, if anything is different, change it in order to do as real HTTP\n\n"

In [3]:
query = "This is a IPv4 TCP-HTTP conversation summary:\n"
query += 'Source: IP="193.24.227.230", port=40301, Window: 4128 // Destination: IP="9.9.9.9", port=80, Window: 256// Others: Host="http://ip.webernetz.net/", Path="", Code=200,  Content_Type="(text/html)"\n\n'

query += "This is the python code to generate the TCP 3 way hanshake packets of the conversation, respectively:\n"
query += "x,y = random.sample(range(1, 100), 2)\n"
query += 'pkt_1= IP(src="193.24.227.230", dst="9.9.9.9")/TCP(sport=40301, dport=80, flags="S", seq=y, window=4128)\n'
query += 'time.sleep(abs(random.gauss(0, 0.03)))\n'
query += 'y= y+len(pkt_1[TCP].payload)+1\n'
query += 'pkt_2= IP(src="9.9.9.9", dst="193.24.227.230")/TCP(sport=80, dport=40301, flags="SA", seq=x, ack=y, window=256)\n'
query += 'time.sleep(abs(random.gauss(0, 0.03)))\n'
query += 'y= y+len(pkt_1[TCP].payload)+1\n'
query += 'pkt_3= IP(src="193.24.227.230", dst="9.9.9.9")/TCP(sport=40301, dport=80, flags="SA", seq=y+len(pkt_1[TCP].payload), ack=x+len(pkt_2[TCP].payload)+1, window=4128)\n\n'
query += 'time.sleep(abs(random.gauss(0, 0.03)))\n'

query += "This part between the handshake and the teardown must be filled with bidirectional realistic HTTP traffic, here is an example of these packets:\n"
query += 'pkt_4= IP(src="193.24.227.230",  dst="9.9.9.9") / TCP(sport=40301, dport=80, flags="PA", seq=y, ack=x+1, window=8192) / HTTP() / HTTPRequest(Method ="GET", Http_Version="HTTP/1.1", Host="ip.webernetz.net", Path="/")\n'
query += 'time.sleep(abs(random.gauss(0, 0.03)))\n'
query += 'pkt_5= IP(src="193.24.227.230", dst="9.9.9.9")/TCP(sport=40301, dport=53, flags="A", seq=x, ack=y+len(pkt_4[TCP].payload), window=4128)\n\n'
query += 'time.sleep(abs(random.gauss(0, 0.03)))\n'
query += 'pkt_6= IP(src="9.9.9.9",  dst="193.24.227.230") / TCP(sport=80, dport=40301, flags="PA", seq=x+1, ack=y+len(pkt_4[TCP].payload)+len(pkt_4[TCP].payload), window=8192) / HTTP() / HTTPResponse(Http_Version= "HTTP/1.1", Status_Code= "200", Reason_Phrase= "OK", Content_Type="text/html") / "<HTML><BODY><H!>Hello World!</H1></BODY></HTML>"\n'
query += 'time.sleep(abs(random.gauss(0, 0.03)))\n'
query += 'pkt_7= IP(src="193.24.227.230", dst="9.9.9.9")/TCP(sport=40301, dport=53, flags="A", seq=K, ack=L+1, window=4128)\n\n'
query += 'time.sleep(abs(random.gauss(0, 0.03)))\n'

query += 'pkt_8= IP(src="193.24.227.230", dst="9.9.9.9")/TCP(sport=40301, dport=80, flags="FA", seq=K, ack=L+ len(pkt_5[TCP].payload) +1, window=4128)\n'
query += 'time.sleep(abs(random.gauss(0, 0.03)))\n'
query += 'pkt_9= IP(src="9.9.9.9", dst="193.24.227.230")/TCP(sport=80, dport=40301, flags="A", seq=L, ack=K+1, window=256)\n'
query += 'time.sleep(abs(random.gauss(0, 0.03)))\n'
query += 'pkt_10= IP(src="9.9.9.9", dst="193.24.227.230")/TCP(sport=80, dport=40301, flags="FA", seq=L, ack=K+1, window=256)\n'
query += 'time.sleep(abs(random.gauss(0, 0.03)))\n'
query += 'pkt_11= IP(src="193.24.227.230", dst="9.9.9.9")/TCP(sport=40301, dport=53, flags="A", seq=K, ack=L+1, window=4128)\n\n'

query += "Variable names in the HTTP definition are extremely important, use the exact same word. For example, use Http_Version, not http_version, use Method, not method.\n\n"
query += "Use the same dport for HTTPRequest (dport=80) and sport for HTTPResponse (sport=80), as this means we are creating a HTTP conection and using another port like 22 might make it seem like SSH traffic\n\n"

query += 'Substitute variables x, y with random numbers but they must be the same along the conversation\n'
query += 'Variables K and L are critical and must be continuous with seq and ack numbers before them\n'

query += 'Pay special attention to the sequence and ack numbers so they make sense for a TCP conversation. The must be continuous throgh the conversation. Here you have a little explanation of how they work:\n'
query += 'The Sequence Number (SEQ) marks the starting byte of each segment sent. It increments by the number of bytes in the payload of each outgoing segment, reflecting the continuous flow of data.In the HTTP packets this payload represents the length of the HTTP Request and HTTP Response, this is described in the variables len(pkt_4[TCP].payload) and len(pkt_5[TCP].payload). Maintain this part in the commands as it is. This is ESSENTIAL\n'
query += 'The Acknowledgment Number (ACK) acknowledges the receipt of data. It corresponds to the next expected byte the receiver anticipates from the sender. As segments arrive at the receiver, the ACK number adjusts to reflect the highest contiguous sequence number received.\n'
query += 'So, SEQ numbers increase with outgoing data, while ACK numbers advance with incoming acknowledgments, ensuring synchronized data exchange in a TCP connection.\n\n'

query += "Take your time (few seconds) to generate the commands and compare your proposed solution with real HTTP functioning, if anything is different, change it in order to do as real HTTP\n\n"

query += 'You must also take into account the size of the TCP payload (in Bytes) in order to calculate the next ACK and SEQ number. In the HTTP packets this payload represents the length of the HTTP Request and HTTP Response, this is described in the variables len(pkt_4[TCP].payload) and len(pkt_5[TCP].payload). Maintain this part in the commands. This is ESSENTIAL.\n\n'

## Step 2: Carga de posibles conversaciones
---

In [4]:
with open("../../data/new/HTTP_Conv_Testing_v2.txt","r",encoding="utf8") as f:
    Conv_summaries = f.read().splitlines()
print("Number of conversation summaries: " + str(len(Conv_summaries)))

Number of conversation summaries: 148


### 2.1 Carga de posibles conversaciones anteriores para concatenarlas

In [5]:
class response():
    "Stores name and place pairs"
    def __init__(self, name, place):
        self.prompt_summary = name
        self.completion = place

In [6]:
try:
    responses = pickle.load( open("../../data/Conversations/HTTP/pickle/HTTP_pairs_Aday_v3.pkl", "rb" ))
except:
    responses = []
    
print("Number of previous responses: " + str(len(responses)))

Number of previous responses: 0


## Step 3: Generación de comandos de paquetes conversaciones TCP-HTTP
---

In [7]:
i=0
for i in range(100):
    
    print("...........................................")
    print( " GENERATING SCAPY COMMANDS FOR THE FOLLOWING CONVERSATIONS (iter " +str(i+1) +")")
    

    ## Luego seleccionas aleatoriamente resumenes ya generados para pedirle que te saque el comando a partir del paquete
    summary = Conv_summaries[i]
    text_sum = ""
    
    
    query_content = "This is a conversation summary:\n"
    query_content += summary + "\n"
    text_sum += summary + "\n"
    query_content += "\nGenerate python code for creating the packets in the conversation with scapy framework.Don´t give back any comments, just bare code. PLEASE DONT start the resposnses wiht ```python This is a flagrant error and will make the code unexecutable. You have done this last step wrong plenty of times, take your time to generate the output withount ```python please.\n"

    #Query es el mensaje que le envias al modelo
    print(query_content)

    messages=[{"role": "system", "content": system_message},
        {"role": "user", "content": query_content}]
    
    #Esta es la petición que le mandas al modelo de IA
    completion = client.chat.completions.create(
        model="gpt-4o-mini", 
        messages=messages,
        max_tokens=2600,
        temperature=0.1)
    
    
    print("...Generated!")
    
    
    # Guardamos tanto el resumen de los paquetes que solicitamos y lo que devuelve el modelo para estos paquetes
    responses.append(response(text_sum, completion))
    
    
    codigo = responses[-1].completion.choices[0].message.content
    
    if codigo.startswith("```python"):
        codigo = codigo[len("```python"):].lstrip()  # Elimina ```python al inicio
    if codigo.endswith("```"):
        codigo = codigo[:-len("```")].rstrip()  # Elimina ``` al final
    
    exec(codigo)
        
    pkt_list = [pkt_1, pkt_2, pkt_3, pkt_4, pkt_5, pkt_6, pkt_7, pkt_8, pkt_9, pkt_10, pkt_11]
    
    
    
    # Guardamos los paquetes generados en un pcap
    with open("../../data/Conversations/HTTP/pcap/HTTP_generated_gpt_4omini_v3.pcap", "ba+") as f:
        wrpcap(f, pkt_list, append=True)
        
    # Guardamos tanto la petición como la respuesta en un archivo
    pickle.dump(responses, open( "../../data/Conversations/HTTP/pickle/HTTP_pairs_Aday_v3.pkl", "wb" ) )
    
    print("Number of completitions done: " + str(len(responses)))

...........................................
 GENERATING SCAPY COMMANDS FOR THE FOLLOWING CONVERSATIONS (iter 1)
This is a conversation summary:
Source: IP="10.0.0.200", port=9352, Window: 8192 // Destination: IP="10.0.0.201", port=80, Window: 1384 // Others: Host="http://www.adobe.com/", Path="", Code=200,  Reason_Phrase= "OK"

Generate python code for creating the packets in the conversation with scapy framework.Don´t give back any comments, just bare code. PLEASE DONT start the resposnses wiht ```python This is a flagrant error and will make the code unexecutable. You have done this last step wrong plenty of times, take your time to generate the output withount ```python please.

...Generated!
Number of completitions done: 1
...........................................
 GENERATING SCAPY COMMANDS FOR THE FOLLOWING CONVERSATIONS (iter 2)
This is a conversation summary:
Source: IP="52.84.125.96", port=5041, Window: 64321 // Destination: IP="13.33.23.213", port=80, Window: 32160 // Others

Number of completitions done: 11
...........................................
 GENERATING SCAPY COMMANDS FOR THE FOLLOWING CONVERSATIONS (iter 12)
This is a conversation summary:
Source: IP="23.4.8.12", port=1052, Window: 15148 // Destination: IP="47.54.62.70", port=80, Window: 15164 // Others: Host="http://www.amazon.co.uk/", Path="", Code=307, Reason_Phrase= "Temporary Redirect"

Generate python code for creating the packets in the conversation with scapy framework.Don´t give back any comments, just bare code. PLEASE DONT start the resposnses wiht ```python This is a flagrant error and will make the code unexecutable. You have done this last step wrong plenty of times, take your time to generate the output withount ```python please.

...Generated!
Number of completitions done: 12
...........................................
 GENERATING SCAPY COMMANDS FOR THE FOLLOWING CONVERSATIONS (iter 13)
This is a conversation summary:
Source: IP="23.5.10.15", port=1059, Window: 15185 // Destinatio

...Generated!
Number of completitions done: 22
...........................................
 GENERATING SCAPY COMMANDS FOR THE FOLLOWING CONVERSATIONS (iter 23)
This is a conversation summary:
Source: IP="23.15.30.45", port=1129, Window: 15555 // Destination: IP="47.65.95.125", port=80, Window: 15615 // Others: Host="http://www.linkedin.com/", Path="", Code=403, Reason_Phrase= "Forbidden"

Generate python code for creating the packets in the conversation with scapy framework.Don´t give back any comments, just bare code. PLEASE DONT start the resposnses wiht ```python This is a flagrant error and will make the code unexecutable. You have done this last step wrong plenty of times, take your time to generate the output withount ```python please.

...Generated!
Number of completitions done: 23
...........................................
 GENERATING SCAPY COMMANDS FOR THE FOLLOWING CONVERSATIONS (iter 24)
This is a conversation summary:
Source: IP="23.16.32.48", port=1136, Window: 15592 // D

...Generated!
Number of completitions done: 33
...........................................
 GENERATING SCAPY COMMANDS FOR THE FOLLOWING CONVERSATIONS (iter 34)
This is a conversation summary:
Source: IP="23.26.52.78", port=1206, Window: 15962 // Destination: IP="47.76.128.180", port=80, Window: 16066 // Others: Host="http://www.amazon.de/", Path="", Code=404, Reason_Phrase= "Not Found"

Generate python code for creating the packets in the conversation with scapy framework.Don´t give back any comments, just bare code. PLEASE DONT start the resposnses wiht ```python This is a flagrant error and will make the code unexecutable. You have done this last step wrong plenty of times, take your time to generate the output withount ```python please.

...Generated!
Number of completitions done: 34
...........................................
 GENERATING SCAPY COMMANDS FOR THE FOLLOWING CONVERSATIONS (iter 35)
This is a conversation summary:
Source: IP="23.27.54.81", port=1213, Window: 15999 // Des

...Generated!
Number of completitions done: 44
...........................................
 GENERATING SCAPY COMMANDS FOR THE FOLLOWING CONVERSATIONS (iter 45)
This is a conversation summary:
Source: IP="23.37.74.111", port=1283, Window: 16369 // Destination: IP="47.87.161.235", port=80, Window: 16517 // Others: Host="http://www.apple.com/", Path="", Code=200, Reason_Phrase= "OK"

Generate python code for creating the packets in the conversation with scapy framework.Don´t give back any comments, just bare code. PLEASE DONT start the resposnses wiht ```python This is a flagrant error and will make the code unexecutable. You have done this last step wrong plenty of times, take your time to generate the output withount ```python please.

...Generated!
Number of completitions done: 45
...........................................
 GENERATING SCAPY COMMANDS FOR THE FOLLOWING CONVERSATIONS (iter 46)
This is a conversation summary:
Source: IP="23.38.76.114", port=1290, Window: 16406 // Destinat

...Generated!
Number of completitions done: 55
...........................................
 GENERATING SCAPY COMMANDS FOR THE FOLLOWING CONVERSATIONS (iter 56)
This is a conversation summary:
Source: IP="23.48.96.144", port=1360, Window: 16776 // Destination: IP="47.98.195.34", port=80, Window: 16968 // Others: Host="http://www.amazon.ca/", Path="", Code=307, Reason_Phrase= "Temporary Redirect"

Generate python code for creating the packets in the conversation with scapy framework.Don´t give back any comments, just bare code. PLEASE DONT start the resposnses wiht ```python This is a flagrant error and will make the code unexecutable. You have done this last step wrong plenty of times, take your time to generate the output withount ```python please.

...Generated!
Number of completitions done: 56
...........................................
 GENERATING SCAPY COMMANDS FOR THE FOLLOWING CONVERSATIONS (iter 57)
This is a conversation summary:
Source: IP="23.49.98.147", port=1367, Window: 16

...Generated!
Number of completitions done: 66
...........................................
 GENERATING SCAPY COMMANDS FOR THE FOLLOWING CONVERSATIONS (iter 67)
This is a conversation summary:
Source: IP="23.59.118.177", port=1437, Window: 17183 // Destination: IP="47.109.228.89", port=80, Window: 17419 // Others: Host="http://www.yahoo.com/", Path="", Code=403, Reason_Phrase= "Forbidden"

Generate python code for creating the packets in the conversation with scapy framework.Don´t give back any comments, just bare code. PLEASE DONT start the resposnses wiht ```python This is a flagrant error and will make the code unexecutable. You have done this last step wrong plenty of times, take your time to generate the output withount ```python please.

...Generated!
Number of completitions done: 67
...........................................
 GENERATING SCAPY COMMANDS FOR THE FOLLOWING CONVERSATIONS (iter 68)
This is a conversation summary:
Source: IP="23.60.120.180", port=1444, Window: 17220 //

...Generated!
Number of completitions done: 77
...........................................
 GENERATING SCAPY COMMANDS FOR THE FOLLOWING CONVERSATIONS (iter 78)
This is a conversation summary:
Source: IP="23.70.140.210", port=1514, Window: 17590 // Destination: IP="47.120.6.144", port=80, Window: 17870 // Others: Host="http://www.facebook.com/", Path="", Code=404, Reason_Phrase= "Not Found"

Generate python code for creating the packets in the conversation with scapy framework.Don´t give back any comments, just bare code. PLEASE DONT start the resposnses wiht ```python This is a flagrant error and will make the code unexecutable. You have done this last step wrong plenty of times, take your time to generate the output withount ```python please.

...Generated!
Number of completitions done: 78
...........................................
 GENERATING SCAPY COMMANDS FOR THE FOLLOWING CONVERSATIONS (iter 79)
This is a conversation summary:
Source: IP="23.71.142.213", port=1521, Window: 17627 

...Generated!
Number of completitions done: 88
...........................................
 GENERATING SCAPY COMMANDS FOR THE FOLLOWING CONVERSATIONS (iter 89)
This is a conversation summary:
Source: IP="23.81.162.243", port=1591, Window: 17997 // Destination: IP="47.131.39.199", port=80, Window: 18321 // Others: Host="http://www.adobe.com/", Path="", Code=200, Reason_Phrase= "OK"

Generate python code for creating the packets in the conversation with scapy framework.Don´t give back any comments, just bare code. PLEASE DONT start the resposnses wiht ```python This is a flagrant error and will make the code unexecutable. You have done this last step wrong plenty of times, take your time to generate the output withount ```python please.

...Generated!
Number of completitions done: 89
...........................................
 GENERATING SCAPY COMMANDS FOR THE FOLLOWING CONVERSATIONS (iter 90)
This is a conversation summary:
Source: IP="23.82.164.246", port=1598, Window: 18034 // Destin

...Generated!
Number of completitions done: 99
...........................................
 GENERATING SCAPY COMMANDS FOR THE FOLLOWING CONVERSATIONS (iter 100)
This is a conversation summary:
Source: IP="23.92.184.186", port=1668, Window: 18404 // Destination: IP="47.142.73.4", port=80, Window: 18772 // Others: Host="http://www.netflix.com/", Path="", Code=307, Reason_Phrase= "Temporary Redirect"

Generate python code for creating the packets in the conversation with scapy framework.Don´t give back any comments, just bare code. PLEASE DONT start the resposnses wiht ```python This is a flagrant error and will make the code unexecutable. You have done this last step wrong plenty of times, take your time to generate the output withount ```python please.

...Generated!
Number of completitions done: 100


In [51]:
print(responses[-1].completion.choices[0].message.content[9:-4])


from scapy.all import *
import random
import time

x, y = random.sample(range(1, 100), 2)

pkt_1 = IP(src="205.251.242.103", dst="151.101.0.223") / TCP(sport=5021, dport=80, flags="S", seq=y, window=23456)
time.sleep(abs(random.gauss(0, 0.03)))
y += len(pkt_1[TCP].payload) + 1

pkt_2 = IP(src="151.101.0.223", dst="205.251.242.103") / TCP(sport=80, dport=5021, flags="SA", seq=x, ack=y, window=11728)
time.sleep(abs(random.gauss(0, 0.03)))
x += len(pkt_2[TCP].payload) + 1

pkt_3 = IP(src="205.251.242.103", dst="151.101.0.223") / TCP(sport=5021, dport=80, flags="A", seq=y, ack=x, window=23456)
time.sleep(abs(random.gauss(0, 0.03)))
y += len(pkt_3[TCP].payload)

pkt_4 = IP(src="205.251.242.103", dst="151.101.0.223") / TCP(sport=5021, dport=80, flags="PA", seq=y, ack=x, window=8192) / HTTP() / HTTPRequest(Method="GET", Http_Version="HTTP/1.1", Host="www.imgur.com", Path="/")
time.sleep(abs(random.gauss(0, 0.03)))
y += len(pkt_4[TCP].payload)

pkt_5 = IP(src="151.101.0.223", dst="205.251.242

responses[-1].completion.choices[0].message.content[9:-4]## Step 4: Estudio de los resultados
---

In [256]:
# Este es el resumen de lo que está en el struct de responses para la última petición
print(responses[-1].prompt_summary)
print(responses[-1].completion.choices[0].text)

Source: IP="192.168.0.100", port=8080, Window: 3768 // Destination: IP="192.168.0.101", port=80, Window: 6536 // Others: Host="http://www.twitter.com/", Path="", Code=200,  Reason_Phrase= "OK"


x,y = random.sample(range(1, 100), 2)
pkt_1= IP(src="192.168.0.100", dst="192.168.0.101")/TCP(sport=8080, dport=80, flags="S", seq=y, window=3768)
time.sleep(abs(random.gauss(0, 0.03)))
y += len(pkt_1[TCP].payload)+1
pkt_2= IP(src="192.168.0.101", dst="192.168.0.100")/TCP(sport=80, dport=8080, flags="SA", seq=x, ack=y, window=6536)
time.sleep(abs(random.gauss(0, 0.03)))
x += len(pkt_2[TCP].payload)+1
pkt_3= IP(src="192.168.0.100", dst="192.168.0.101")/TCP(sport=8080, dport=80, flags="A", seq=y, ack=x, window=3768)

time.sleep(abs(random.gauss(0, 0.03)))
y += len(pkt_3[TCP].payload)
This part between the handshake and the teardown must be filled with bidirectional realistic HTTP traffic, here is an example of these packets:
pkt_4= IP(src="192.168.0.100",  dst="192.168.0.101") / TCP(sport=8080, d

### 4.1 Generación de paquetes

In [211]:
# Ejecutamos los comandos que devuelve el modelo para generar los paquetes
exec(responses[-1].completion.choices[0].text)
pkt_list = [pkt_1, pkt_2, pkt_3, pkt_4, pkt_5, pkt_6, pkt_7, pkt_8, pkt_9, pkt_10, pkt_11]
print(pkt_list)

[<IP  frag=0 proto=tcp src=10.0.0.200 dst=10.0.0.201 |<TCP  sport=9352 dport=http seq=99 flags=S window=8192 |>>, <IP  frag=0 proto=tcp src=10.0.0.201 dst=10.0.0.200 |<TCP  sport=http dport=9352 seq=6 ack=100 flags=SA window=1384 |>>, <IP  frag=0 proto=tcp src=10.0.0.200 dst=10.0.0.201 |<TCP  sport=9352 dport=http seq=100 ack=7 flags=A window=8192 |>>, <IP  frag=0 proto=tcp src=10.0.0.200 dst=10.0.0.201 |<TCP  sport=9352 dport=http seq=100 ack=7 flags=PA window=8192 |<HTTP  |<HTTPRequest  Method='GET' Path='/' Http_Version='HTTP/1.1' Host='ip.webernetz.net' |>>>>, <IP  frag=0 proto=tcp src=10.0.0.201 dst=10.0.0.200 |<TCP  sport=http dport=9352 seq=7 ack=142 flags=A window=1384 |>>, <IP  frag=0 proto=tcp src=10.0.0.201 dst=10.0.0.200 |<TCP  sport=http dport=9352 seq=7 ack=142 flags=PA window=8192 |<HTTP  |<HTTPResponse  Http_Version='HTTP/1.1' Status_Code='200' Reason_Phrase='OK' Content_Type='text/html' |<Raw  load='<HTML><BODY><H1>Hello World!</H1></BODY></HTML>' |>>>>>, <IP  frag=0 p

In [213]:
#Enviarlos para probar si se envian correctamente
send(pkt_list)


Sent 11 packets.


## Step 5: Introducción en un pcap
---

In [215]:
# Guardamos los paquetes generados en un pcap
with open("../../data/Conversations/HTTP/pcap/HTTP_Conv2_generated_3_5_turbo-instruct_conv_ALL.pcap", "ba+") as f:
    wrpcap(f, pkt_list, append=True)

In [158]:
query_p = "x,y = random.sample(range(1, 100), 2)\n"
query_p += 'print(y)\n'
query_p += 'pkt_1= IP(src="193.24.227.230", dst="9.9.9.9")/TCP(sport=40301, dport=80, flags="S", seq=y, window=4128)\n'
query_p += 'time.sleep(abs(random.gauss(0, 0.03)))\n'
query_p += 'y += len(pkt_1[TCP].payload)+1\n'
query_p += 'pkt_2= IP(src="9.9.9.9", dst="193.24.227.230")/TCP(sport=80, dport=40301, flags="SA", seq=x, ack=y, window=256)\n'
query_p += 'time.sleep(abs(random.gauss(0, 0.03)))\n'
query_p += 'x += len(pkt_2[TCP].payload)+1\n'
query_p += 'pkt_3= IP(src="193.24.227.230", dst="9.9.9.9")/TCP(sport=40301, dport=80, flags="A", seq=y, ack=x, window=4128)\n\n'
query_p += 'time.sleep(abs(random.gauss(0, 0.03)))\n'
query_p += 'y += len(pkt_3[TCP].payload)\n'

query_p += 'pkt_4= IP(src="193.24.227.230",  dst="9.9.9.9") / TCP(sport=40301, dport=80, flags="PA", seq=y, ack=x, window=8192) / HTTP() / HTTPRequest(Method ="GET", Http_Version="HTTP/1.1", Host="ip.webernetz.net", Path="/")\n'
query_p += 'time.sleep(abs(random.gauss(0, 0.03)))\n'
query_p += 'y += len(pkt_4[TCP].payload)\n'
query_p += 'print(y)\n'
query_p += 'pkt_5= IP(src="9.9.9.9", dst="193.24.227.230")/TCP(sport=80, dport=40301, flags="A", seq=x, ack=y, window=4128)\n\n'
query_p += 'time.sleep(abs(random.gauss(0, 0.03)))\n'
query_p += 'x += len(pkt_5[TCP].payload)\n'
query_p += 'pkt_6= IP(src="9.9.9.9",  dst="193.24.227.230") / TCP(sport=80, dport=40301, flags="PA", seq=x, ack=y, window=8192) / HTTP() / HTTPResponse(Http_Version= "HTTP/1.1", Status_Code= "200", Reason_Phrase= "OK", Content_Type="text/html") / "<HTML><BODY><H!>Hello World!</H1></BODY></HTML>"\n'
query_p += 'time.sleep(abs(random.gauss(0, 0.03)))\n'
query_p += 'x += len(pkt_6[TCP].payload)\n'
query_p += 'pkt_7= IP(src="193.24.227.230", dst="9.9.9.9")/TCP(sport=40301, dport=80, flags="A", seq=y, ack=x, window=4128)\n\n'
query_p += 'time.sleep(abs(random.gauss(0, 0.03)))\n'
query_p += 'y += len(pkt_7[TCP].payload)\n'

query_p += 'pkt_8= IP(src="193.24.227.230", dst="9.9.9.9")/TCP(sport=40301, dport=80, flags="FA", seq=y, ack=x, window=4128)\n'
query_p += 'time.sleep(abs(random.gauss(0, 0.03)))\n'
query_p += 'y += len(pkt_8[TCP].payload)\n'
query_p += 'pkt_9= IP(src="9.9.9.9", dst="193.24.227.230")/TCP(sport=80, dport=40301, flags="A", seq=x, ack=y, window=256)\n'
query_p += 'time.sleep(abs(random.gauss(0, 0.03)))\n'
query_p += 'x += len(pkt_9[TCP].payload)\n'
query_p += 'pkt_10= IP(src="9.9.9.9", dst="193.24.227.230")/TCP(sport=80, dport=40301, flags="FA", seq=x, ack=y, window=256)\n'
query_p += 'time.sleep(abs(random.gauss(0, 0.03)))\n'
query_p += 'x += len(pkt_10[TCP].payload)\n'
query_p += 'pkt_11= IP(src="193.24.227.230", dst="9.9.9.9")/TCP(sport=40301, dport=80, flags="A", seq=y, ack=x, window=4128)\n\n'
query_p += 'y += len(pkt_11[TCP].payload)\n'

exec(query_p)

In [170]:
query_p = "x,y = random.sample(range(1, 100), 2)\n"
query_p += 'print(y)\n'
query_p += 'pkt_1= IP(src="193.24.227.230", dst="9.9.9.9")/TCP(sport=40301, dport=80, flags="S", seq=y, window=4128)\n'
query_p += 'time.sleep(abs(random.gauss(0, 0.03)))\n'
query_p += 'y += len(pkt_1[TCP].payload)+1\n'
query_p += 'pkt_2= IP(src="9.9.9.9", dst="193.24.227.230")/TCP(sport=80, dport=40301, flags="SA", seq=x, ack=y, window=256)\n'
query_p += 'time.sleep(abs(random.gauss(0, 0.03)))\n'
query_p += 'x += len(pkt_2[TCP].payload)+1\n'
query_p += 'pkt_3= IP(src="193.24.227.230", dst="9.9.9.9")/TCP(sport=40301, dport=80, flags="A", seq=y, ack=x, window=4128)\n\n'
query_p += 'time.sleep(abs(random.gauss(0, 0.03)))\n'
query_p += 'y += len(pkt_3[TCP].payload)\n'

query_p += 'pkt_4= IP(src="193.24.227.230",  dst="9.9.9.9") / TCP(sport=40301, dport=80, flags="PA", seq=y, ack=x, window=8192) / HTTP() / HTTPRequest(Method ="GET", Http_Version="HTTP/1.1", Host="ip.webernetz.net", Path="/")\n'
query_p += 'time.sleep(abs(random.gauss(0, 0.03)))\n'
query_p += 'y += len(pkt_4[TCP].payload)\n'
query_p += 'print(y)\n'
query_p += 'pkt_5= IP(src="9.9.9.9", dst="193.24.227.230")/TCP(sport=80, dport=40301, flags="A", seq=x, ack=y, window=4128)\n\n'
query_p += 'time.sleep(abs(random.gauss(0, 0.03)))\n'
query_p += 'x += len(pkt_5[TCP].payload)\n'
query_p += 'pkt_6= IP(src="9.9.9.9",  dst="193.24.227.230") / TCP(sport=80, dport=40301, flags="PA", seq=x, ack=y, window=8192) / HTTP() / HTTPResponse(Http_Version= "HTTP/1.1", Status_Code= "200", Reason_Phrase= "OK", Content_Type="text/html") / "<HTML><BODY><H!>Hello World!</H1></BODY></HTML>"\n'
query_p += 'time.sleep(abs(random.gauss(0, 0.03)))\n'
query_p += 'x += len(pkt_6[TCP].payload)\n'
query_p += 'pkt_7= IP(src="193.24.227.230", dst="9.9.9.9")/TCP(sport=40301, dport=80, flags="A", seq=y, ack=x, window=4128)\n\n'
query_p += 'time.sleep(abs(random.gauss(0, 0.03)))\n'
query_p += 'y += len(pkt_7[TCP].payload)\n'

query_p += 'pkt_8= IP(src="9.9.9.9", dst="193.24.227.230")/TCP(sport=80, dport=40301, flags="FA", seq=x, ack=y, window=256)\n'
query_p += 'time.sleep(abs(random.gauss(0, 0.03)))\n'
query_p += 'x += len(pkt_8[TCP].payload)\n'
query_p += 'pkt_9= IP(src="193.24.227.230", dst="9.9.9.9")/TCP(sport=40301, dport=80, flags="A", seq=y, ack=x, window=4128)\n\n'
query_p += 'y += len(pkt_9[TCP].payload)\n'
query_p += 'pkt_10= IP(src="193.24.227.230", dst="9.9.9.9")/TCP(sport=40301, dport=80, flags="FA", seq=y, ack=x, window=4128)\n'
query_p += 'time.sleep(abs(random.gauss(0, 0.03)))\n'
query_p += 'y += len(pkt_10[TCP].payload)\n'
query_p += 'pkt_11= IP(src="9.9.9.9", dst="193.24.227.230")/TCP(sport=80, dport=40301, flags="A", seq=x, ack=y, window=256)\n'
query_p += 'time.sleep(abs(random.gauss(0, 0.03)))\n'
query_p += 'x += len(pkt_11[TCP].payload)\n'

exec(query_p)

38
81


In [172]:
# Guardamos los paquetes generados en un pcap
with open("../../data/Conversations/HTTP/pcap/HTTP_Conv2_generated_3_5_turbo-instruct_conv_ALL.pcap", "ba+") as f:
    wrpcap(f, [pkt_1, pkt_2, pkt_3, pkt_4, pkt_5, pkt_6, pkt_7, pkt_8, pkt_9, pkt_10, pkt_11], append=True)

In [140]:
len(pkt_4[TCP].payload)

42